In [1]:
import numpy as np
import pandas as pd
from glob import glob
import librosa as lr
import librosa.display
import IPython.display as ipd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
data_dir = "./Audio"
audio_files = glob(data_dir + "/*.wav")
data = pd.DataFrame()
Emotion = []
Emotional_Intensity = []
Statement = []
Statement_Text = []
Repetition = []
Actor = []

In [3]:
for i in range(len(audio_files)):
    temp = audio_files[i][audio_files[i].index("\\")+1:audio_files[0].index(".wav")].split("-")
    
    # creating Emotion
    if temp[2] == "01":
        Emotion.append(1)
    if temp[2] == "02":
        Emotion.append(2)
    if temp[2] == "03":
        Emotion.append(3)
    if temp[2] == "04":
        Emotion.append(4)
    if temp[2] == "05":
        Emotion.append(5)
    if temp[2] == "06":
        Emotion.append(6)
    if temp[2] == "07":
        Emotion.append(7)
    if temp[2] == "08":
        Emotion.append(8)

   ############################

    # creating Emotinal_Intensity
    if temp[3] == "01":
        Emotional_Intensity.append(1)
    if temp[3] == "02":
        Emotional_Intensity.append(2)
        
   #############################

    # creating Statement
    if temp[4] == "01":
        Statement.append(1)
        Statement_Text.append("Kids are talking by the door")
    if temp[4] == "02":
        Statement.append(2)
        Statement_Text.append("Dogs are sitting by the door")
        
    ############################
    
    # creating Repetition
    if temp[5] == "01":
        Repetition.append(1)
    if temp[5] == "02":
        Repetition.append(2)
        
    ###########################
    
    # creating actor gender
    if int(temp[6])%2 == 0:   # even --> female
        Actor.append(1)
    if int(temp[6])%2 == 1:   # odd --> male
        Actor.append(0)    


In [4]:
data["Emotion"] = Emotion
data["Emotional_Intensity"] = Emotional_Intensity
data["Statement"] = Statement
data["Statement_Text"] = Statement_Text
data["Repetition"] = Repetition
data["Actor_Gender"] = Actor

In [5]:
#extracting mfcc features from audios
mfcc1 = []
for i in range(0,len(audio_files)):
    
    y, sr = lr.load(audio_files[i],res_type="kaiser_fast",duration=3,offset=0.5)
    sr=np.array(sr)
    mfcc=(lr.feature.mfcc(y=y,hop_length=256,n_mfcc=13))
    #This line controls audios lenght if it is shorter than the trashold algoritm pads mfcc feature with 0
    if (259 > mfcc.shape[1]):
        pad_width = 259 - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        mfcc1.append(mfcc)
    else:
        mfcc1.append(mfcc)
    
mfccTarget1 = pd.DataFrame()
mfccTarget1["mfcc"] = mfcc1


In [6]:
#LSTM model with the best accuracies.
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
X_train, X_test, y_train, y_test = train_test_split(np.asarray(mfccTarget1["mfcc"].tolist()),data["Emotion"].to_numpy(), test_size=0.30, random_state=42,shuffle=True)
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(13,259)))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units = 9, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100)

Train on 1008 samples, validate on 432 samples
Epoch 1/100
1008/1008 [==============================] - 5s 5ms/step - loss: 2.0775 - accuracy: 0.1974 - val_loss: 1.9552 - val_accuracy: 0.2245
Epoch 2/100
1008/1008 [==============================] - 2s 2ms/step - loss: 1.8299 - accuracy: 0.2966 - val_loss: 1.8655 - val_accuracy: 0.2593
Epoch 3/100
1008/1008 [==============================] - 2s 2ms/step - loss: 1.7222 - accuracy: 0.3423 - val_loss: 1.8853 - val_accuracy: 0.2407
Epoch 4/100
1008/1008 [==============================] - 2s 2ms/step - loss: 1.6261 - accuracy: 0.3770 - val_loss: 1.8063 - val_accuracy: 0.2685 1.6328 - accuracy: 
Epoch 5/100
1008/1008 [==============================] - 2s 2ms/step - loss: 1.5180 - accuracy: 0.4147 - val_loss: 1.8974 - val_accuracy: 0.2940
Epoch 6/100
1008/1008 [==============================] - 2s 2ms/step - loss: 1.4620 - accuracy: 0.4444 - val_loss: 1.8800 - val_accuracy: 0.2778
Epoch 7/100
1008/1008 [==============================] - 3s 3ms

Epoch 57/100
1008/1008 [==============================] - 1s 1ms/step - loss: 0.2850 - accuracy: 0.9048 - val_loss: 3.0944 - val_accuracy: 0.3148
Epoch 58/100
1008/1008 [==============================] - 2s 2ms/step - loss: 0.2767 - accuracy: 0.9067 - val_loss: 3.0255 - val_accuracy: 0.3495
Epoch 59/100
1008/1008 [==============================] - 2s 2ms/step - loss: 0.2061 - accuracy: 0.9296 - val_loss: 3.1067 - val_accuracy: 0.3449
Epoch 60/100
1008/1008 [==============================] - 2s 2ms/step - loss: 0.2031 - accuracy: 0.9286 - val_loss: 2.9815 - val_accuracy: 0.3542
Epoch 61/100
1008/1008 [==============================] - 2s 2ms/step - loss: 0.1919 - accuracy: 0.9385 - val_loss: 3.0955 - val_accuracy: 0.3565
Epoch 62/100
1008/1008 [==============================] - 2s 2ms/step - loss: 0.2168 - accuracy: 0.9355 - val_loss: 3.0801 - val_accuracy: 0.3681
Epoch 63/100
1008/1008 [==============================] - 2s 2ms/step - loss: 0.2250 - accuracy: 0.9256 - val_loss: 3.0283 -